In [ ]:
%matplotlib inline

from pymongo import MongoClient
import urllib.parse

import os
import sys
import json
from json import dumps

# From within the docker-compose application, use "mongodb_container", for local development use "localhost"
dbServer = os.getenv("MONGO_DB_SERVER", "mongodb_container:27017")
dbUser = os.getenv("MONGO_USERNAME", "root")
dbPW = os.getenv("MONGO_PASSWORD", "rootpassword")

client = MongoClient(dbServer, username=dbUser, password=dbPW)

print("Mongo DB Connection -----")
print("server:" + dbServer)
print("user:" + dbUser)

collection = client.images.images

In [ ]:
def imagelistpage(page_id: int):
    pagesize = 50
    skipoffset = (page_id - 1) * pagesize
    data = []
    for image in (
        collection.find(
            {},
            {
                "datasetprovider": 1,
                "filenameHash": 1,
                "datasetname": 1,
                "imageFilename": 1,
                "timestamp": 1,
                "velocity_lon": 1,
                "yolov5": 1,
                "_id": 0,
            },
        )
        .skip(skipoffset)
        .limit(pagesize)
    ):
        data.append(image)

    returnString = data
    return returnString

imagelistpage(1)

In [ ]:
def find_velocity(min_velocity):
    data = []
    for image in (
        collection.find(
            {"velocity_lon": {"$gt": min_velocity}},
            {
                "datasetprovider": 1,
                "filenameHash": 1,
                "imageFilename": 1,
                "timestamp": 1,
                "velocity_lon": 1,
                "_id": 0,
            },
        )
    ):
        data.append(image)

    returnString = data
    return returnString

find_velocity(12.0)

In [ ]:
def find_yolo(class_label):
    data = []
    for image in (
        collection.find(
            {"yolov5": {"$elemMatch": { "name": class_label}}},
            {
                "datasetname" : 1,
                "filenameHash" : 1,
                "yolov5": 1,
                "_id": 0,
            },
        )
    ):
        data.append(image)

    returnString = data
    return returnString

find_yolo('skateboard')

In [ ]:
def find_complex():
    data = []
    for image in (
        collection.find(
            {"datasetname": {"$in": ['2011_09_26_drive_0002_sync','2011_09_28_drive_0047_sync']}, "yolov5.name": { "$in": ['skateboard','car']}  },
            {
                "datasetname" : 1,
                "filenameHash" : 1,
                "yolov5": 1,
                "_id": 0,
            },
        ).limit(5)
    ):
        data.append(image)

    returnString = data
    return returnString
    
find_complex()

In [ ]:
def distinct_datasetsnames():
    data = []
    result=collection.distinct("datasetname"),
    data.append(result)

    returnString = data
    return returnString

distinct_datasetsnames()

In [ ]:
def distinct_yolo_classes():
    data = []
    result=collection.distinct("yolov5.name"),
    data.append(result)

    returnString = data
    return returnString

distinct_yolo_classes()

In [ ]:
import matplotlib.pyplot as plt

def show_image(filenameHash):
    url = 'http://localhost:8000/imagethumbnail/'+filenameHash
    f = urllib.request.urlopen(url)
    a=plt.imread(f, format='JPG')
    plt.imshow(a)
    
show_image('73b71cfcd213b9f75055dd9e7d327803')


In [ ]:

#var collection = "images"; var field = "datasetname"; 
#db[collection].distinct(field).forEach(function(value){print(field + ", " + value + ": " + db.images.count({[field]: value}))})